In [1]:
import requests
from time import sleep
from bs4 import BeautifulSoup
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')

## Gettings the URLs for all the movies

1. Get the URLs for the major studios.
2. For each major studio, get the URLs for all the pages of movies for that studio.
3. For each page of movies, get the URLs of each individual movie.

In [2]:
response = requests.get('http://www.boxofficemojo.com/studio/?view2=allstudios&view=company&p=.htm')

In [3]:
soup = BeautifulSoup(response.text,"lxml")
tables = soup.find_all('table')

In [9]:
hrefs = list(map((lambda a: a['href']), tables[3].find_all('a')))

In [10]:
hrefs

['chart/?view=company&view2=allmovies&studio=fox.htm',
 'chart/?view=company&view2=allmovies&studio=paramount.htm',
 'chart/?view=company&view2=allmovies&studio=buenavista.htm',
 'chart/?view=company&view2=allmovies&studio=paramountclassics.htm',
 'chart/?view=company&view2=allmovies&studio=dreamworks.htm',
 'chart/?view=company&view2=allmovies&studio=sonyrepertory.htm',
 'chart/?view=company&view2=allmovies&studio=focus.htm',
 'chart/?view=company&view2=allmovies&studio=sonyclassics.htm',
 'chart/?view=company&view2=allmovies&studio=foxsearchlight.htm',
 'chart/?view=company&view2=allmovies&studio=universal.htm',
 'chart/?view=company&view2=allmovies&studio=lionsgate.htm',
 'chart/?view=company&view2=allmovies&studio=warnerbros.htm',
 'chart/?view=company&view2=allmovies&studio=ua.htm',
 'chart/?view=company&view2=allmovies&studio=wip.htm',
 'chart/?view=company&view2=allmovies&studio=miramax.htm',
 'chart/?view=company&view2=allmovies&studio=weinsteincompany.htm',
 'chart/?view=compa

In [11]:
studio_urls = list(map((lambda href: 'http://www.boxofficemojo.com/studio/{}'.format(href)), hrefs))

In [12]:
studio_urls

['http://www.boxofficemojo.com/studio/chart/?view=company&view2=allmovies&studio=fox.htm',
 'http://www.boxofficemojo.com/studio/chart/?view=company&view2=allmovies&studio=paramount.htm',
 'http://www.boxofficemojo.com/studio/chart/?view=company&view2=allmovies&studio=buenavista.htm',
 'http://www.boxofficemojo.com/studio/chart/?view=company&view2=allmovies&studio=paramountclassics.htm',
 'http://www.boxofficemojo.com/studio/chart/?view=company&view2=allmovies&studio=dreamworks.htm',
 'http://www.boxofficemojo.com/studio/chart/?view=company&view2=allmovies&studio=sonyrepertory.htm',
 'http://www.boxofficemojo.com/studio/chart/?view=company&view2=allmovies&studio=focus.htm',
 'http://www.boxofficemojo.com/studio/chart/?view=company&view2=allmovies&studio=sonyclassics.htm',
 'http://www.boxofficemojo.com/studio/chart/?view=company&view2=allmovies&studio=foxsearchlight.htm',
 'http://www.boxofficemojo.com/studio/chart/?view=company&view2=allmovies&studio=universal.htm',
 'http://www.boxof

In [13]:
response = requests.get('http://www.boxofficemojo.com/studio/chart/?view=company&view2=allmovies&studio=fox.htm')

In [14]:
soup = BeautifulSoup(response.text,"lxml")

In [25]:
# main_container = soup.find('div', {'id':'body'})
# container = main_container.find('center')
# hrefs = list(map((lambda a: a['href']), container.find_all('a')))
# hrefs
tables = soup.find_all('table')
rows = tables[2].find_all('tr')[1:]
hrefs = [row.find('a')['href'] for row in rows if row.find('a')]
hrefs

['/movies/?id=avatar.htm',
 '/movies/?id=starwars.htm',
 '/movies/?id=starwars3.htm',
 '/movies/?id=deadpool2016.htm',
 '/movies/?id=starwars4.htm',
 '/movies/?id=independenceday.htm',
 '/movies/?id=starwars2.htm',
 '/movies/?id=homealone.htm',
 '/movies/?id=starwars6.htm',
 '/movies/?id=nightatthemuseum.htm',
 '/movies/?id=x3.htm',
 '/movies/?id=xmen2014.htm',
 '/movies/?id=castaway.htm',
 '/movies/?id=scott2016.htm',
 '/movies/?id=wolverine2017.htm',
 '/movies/?id=alvinandthechipmunksii.htm',
 '/movies/?id=mrsdoubtfire.htm',
 '/movies/?id=alvinandthechipmunks.htm',
 '/movies/?id=x2.htm',
 '/movies/?id=starwars5.htm',
 '/movies/?id=dawnoftheapes.htm',
 '/movies/?id=iceage3.htm',
 '/movies/?id=iceage2.htm',
 '/movies/?id=croods.htm',
 '/movies/?id=dayaftertomorrow.htm',
 '/movies/?id=mrandmrssmith.htm',
 '/movies/?id=revenant.htm',
 '/movies/?id=simpsons.htm',
 '/movies/?id=planetoftheapes01.htm',
 '/movies/?id=wolverine.htm',
 '/movies/?id=happysmekday.htm',
 '/movies/?id=nightatthemu

In [29]:
def get_studio_urls():
    response = requests.get('http://www.boxofficemojo.com/studio/?view2=allstudios&view=company&p=.htm')
    soup = BeautifulSoup(response.text,"lxml")
    tables = soup.find_all('table')
    hrefs = [a['href'] for a in tables[3].find_all('a')]
    studio_urls = ['http://www.boxofficemojo.com/studio/{}'.format(href) for href in hrefs]
    if studio_urls[-1] == 'http://www.boxofficemojo.com/studio/chart/?view=company&view2=allmovies&studio=':
        studio_urls.pop()
    return studio_urls

def get_additional_page_urls_for_studio(studio_url):
    response = requests.get(studio_url)
    soup = BeautifulSoup(response.text,"lxml")
    main_container = soup.find('div', {'id':'body'})
    container = main_container.find('center')
    if not container: return []
    hrefs = [a['href'] for a in container.find_all('a')]
    page_urls = ['http://www.boxofficemojo.com{}'.format(href) for href in hrefs]
    return page_urls

def get_movie_urls_from_studio_page(studio_page_url):
    response = requests.get(studio_page_url)
    soup = BeautifulSoup(response.text,"lxml")
    tables = soup.find_all('table')
    rows = tables[2].find_all('tr')[1:]
    hrefs = [row.find('a')['href'] for row in rows if row.find('a')]
    movie_urls = ['http://www.boxofficemojo.com{}'.format(href) for href in hrefs]
    return movie_urls

In [30]:
studio_urls = get_studio_urls()
studio_page_urls = []
for studio_url in studio_urls:
    sleep(0.2)
    studio_page_urls.append(studio_url)
    studio_page_urls += get_additional_page_urls_for_studio(studio_url)

In [32]:
len(studio_page_urls)

78

In [33]:
movie_urls = []
for studio_page_url in studio_page_urls:
    sleep(0.2)
    movie_urls += get_movie_urls_from_studio_page(studio_page_url)

In [34]:
len(movie_urls)

6928